In [1]:
#path = '/home/arun/Downloads/IfcOpenHouse.ifc'
path = '/home/arun/Downloads/IfcOpenHouse_IFC4.ifc'

In [2]:
import ifcopenshell
import ifcopenshell.geom
import ifcopenshell.util
import ifcopenshell.util.representation
import ifcopenshell.util.element

In [3]:
import OCC.Core.gp
import OCC.Core.Geom

import OCC.Core.Bnd
import OCC.Core.BRepBndLib

import OCC.Core.BRep
import OCC.Core.BRepPrimAPI
import OCC.Core.BRepAlgoAPI
import OCC.Core.BRepBuilderAPI

import OCC.Core.GProp
import OCC.Core.BRepGProp

import OCC.Core.TopoDS
import OCC.Core.TopExp
import OCC.Core.TopAbs

In [4]:
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)

In [5]:
occ_display = ifcopenshell.geom.utils.initialize_display()

qt-pyside2 backend - Qt version 5.15.8
####### 3D rendering pipe initialisation #####
Display3d class initialization starting ...
Aspect_DisplayConnection created.
OpenGl_GraphicDriver created.
V3d_Viewer created.
AIS_InteractiveContext created.
V3d_View created
Xw_Window created.
TKOpenGl | Type: Other | ID: 0 | Severity: Medium | Message:
  OpenGl_Window::CreateWindow: window Visual is incomplete: no stencil buffer


In [6]:
f = ifcopenshell.open(path)
products = f.by_type('IfcProduct')

In [7]:
for p in products:
    if p.is_a('IfcOpeningElement'): continue
    if p.Representation:
        shape = ifcopenshell.geom.create_shape(settings, p).geometry
        display_shape = ifcopenshell.geom.utils.display_shape(shape)
        if p.is_a('IfcPlate'):
            ifcopenshell.geom.utils.set_shape_transparency(ais=display_shape, t=0.8)

In [8]:
    #input()
occ_display.EraseAll()

In [9]:
walls = f.by_type('IfcWall')
wall_shapes = []
bbox = OCC.Core.Bnd.Bnd_Box()

In [10]:
for wall in walls:
    shape = ifcopenshell.geom.create_shape(settings, wall).geometry
    wall_shapes.append((wall, shape))
    OCC.Core.BRepBndLib.brepbndlib_Add(shape, bbox)
    ifcopenshell.geom.utils.display_shape(shape)

bbox_center = ifcopenshell.geom.utils.get_bounding_box_center(bbox)
print(f"Bounding Box Center: {bbox_center.X()}, {bbox_center.Y()}, {bbox_center.Z()}")
occ_display.DisplayMessage(bbox_center, "Center", update=True)

Bounding Box Center: 0.0, 2.5000000000000004, 2.7500000000000013


<class 'Graphic3d_Structure'>

In [11]:
occ_display.EraseAll()

In [12]:
occ_display.DisplayMessage(bbox_center, 'Center', update=True)

<class 'Graphic3d_Structure'>

In [13]:
half_spaces = []
for wall, shape in wall_shapes:
    exp = OCC.Core.TopExp.TopExp_Explorer(shape, OCC.Core.TopAbs.TopAbs_FACE)
    while exp.More():
        face = OCC.Core.TopoDS.topods.Face(exp.Current())
        exp.Next()
        surf = OCC.Core.BRep.BRep_Tool.Surface(face)
        assert surf.DynamicType().Name() == 'Geom_Plane'

        plane = OCC.Core.Geom.Handle_Geom_Plane_DownCast(surf)

        if plane.Axis().Direction().Z() == 0:
            face_bbox = OCC.Core.Bnd.Bnd_Box()
            OCC.Core.BRepBndLib.brepbndlib_Add(face, face_bbox)
            face_center = ifcopenshell.geom.utils.get_bounding_box_center(face_bbox).XYZ()

            face_normal = plane.Axis().Direction().XYZ()
            face_towards_center = bbox_center.XYZ() - face_center
            face_towards_center.Normalize()

            dot = face_towards_center.Dot(face_normal)

            if dot < -0.8:
                ifcopenshell.geom.utils.display_shape(face)
                face_plane = plane.Pln()
                new_face = OCC.Core.BRepBuilderAPI.BRepBuilderAPI_MakeFace(face_plane).Face()
                halfspace = OCC.Core.BRepPrimAPI.BRepPrimAPI_MakeHalfSpace(new_face, bbox_center).Solid()
                half_spaces.append(halfspace)

In [14]:
occ_display.EraseAll()

In [15]:
roofs = f.by_type('IfcRoof')
for roof in ifcopenshell.util.element.get_decomposition(roofs[0]):
    shape = ifcopenshell.geom.create_shape(settings, roof).geometry
    exp = OCC.Core.TopExp.TopExp_Explorer(shape, OCC.Core.TopAbs.TopAbs_FACE)
    while exp.More():
        face = OCC.Core.TopoDS.topods.Face(exp.Current())
        exp.Next()
        surf = OCC.Core.BRep.BRep_Tool.Surface(face)
        assert surf.DynamicType().Name() == 'Geom_Plane'
        plane = OCC.Core.Geom.Handle_Geom_Plane_DownCast(surf)
        if plane.Axis().Direction().Z() > 0.7:
            face_plane = plane.Pln()
            new_face = OCC.Core.BRepBuilderAPI.BRepBuilderAPI_MakeFace(face_plane).Face()
            halfspace = OCC.Core.BRepPrimAPI.BRepPrimAPI_MakeHalfSpace(new_face, bbox_center).Solid()
            half_spaces.append(halfspace)

In [16]:
common_shape = OCC.Core.BRepPrimAPI.BRepPrimAPI_MakeBox(OCC.Core.gp.gp_Pnt(-10,-10,0), OCC.Core.gp.gp_Pnt(10,10,10)).Solid()

In [17]:
for halfspace in half_spaces:
    common_shape = OCC.Core.BRepAlgoAPI.BRepAlgoAPI_Common(common_shape, halfspace).Shape()

In [18]:
ifcopenshell.geom.utils.display_shape(common_shape)

<class 'AIS_Shape'>

In [19]:
props = OCC.Core.GProp.GProp_GProps()
OCC.Core.BRepGProp.brepgprop_VolumeProperties(shape, props)
print(f"Space Volume: {props.Mass()}")

Space Volume: 10.648801737362337


In [20]:
ifcopenshell.geom.utils.main_loop()

INFO:OCC.Display.qtDisplay:key: code 16777251 not mapped to any function
INFO:OCC.Display.qtDisplay:key: code 16777251 not mapped to any function
INFO:OCC.Display.qtDisplay:key: code 16777251 not mapped to any function
INFO:OCC.Display.qtDisplay:key: code 16777251 not mapped to any function
